# Saliva Processing – App Logs

In [ ]:
from pathlib import Path
import json

import numpy as np
import pandas as pd
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.auto import tqdm

import biopsykit as bp
from biopsykit.utils.dataframe_handling import multi_xs
from biopsykit.carwatch_logs import LogData
from biopsykit.carwatch_logs.log_data import get_logs_for_action
import biopsykit.carwatch_logs.log_actions as log_actions


from carwatch_analysis.datasets import CarWatchDatasetProcessed
from carwatch_analysis.exceptions import AppLogDataNotFoundException
from carwatch_analysis.stats import create_unique_night_id

from carwatch_analysis.data_processing.app_logs import (
    process_app_log_single_subject,
    restructure_sampling_times_dataframe,
    add_naive_times,
    sample_times_long_format
)
from carwatch_analysis.data_processing.saliva import compute_saliva_features

import datetime

%load_ext autoreload
%autoreload 2
%matplotlib widget

## Setup Paths

In [ ]:
deploy_type = "local"

In [ ]:
# build path to data folder
config_dict = json.load(Path("../../../config.json").open(encoding="utf-8"))
base_path = Path(config_dict[deploy_type]["base_path"])

base_path

In [ ]:
dataset = CarWatchDatasetProcessed(base_path)
dataset

## Load Data

### Self-Report Wakeup Onsets

In [ ]:
bedtimes = dataset.endpoints_selfreport
bedtimes = bedtimes.dropna().sort_index()
bedtimes = bedtimes[["wake_onset_selfreport"]].apply(pd.to_timedelta)

bedtimes

### App Wakeup Onsets

In [ ]:
app_wakeup_path = base_path.joinpath("app_logs/app_data_wakeup.xlsx")

app_wakeup = pd.read_excel(app_wakeup_path)
app_wakeup = app_wakeup.set_index("Code")
app_wakeup.index.name = "subject"
app_wakeup.columns.name = "night"
app_wakeup = pd.DataFrame(pd.to_timedelta(app_wakeup.stack()), columns=["wake_onset_app"]).sort_index()

app_wakeup

### Cortisol Samples

In [ ]:
cortisol_samples = dataset.cortisol_samples

cortisol_samples = cortisol_samples.rename(columns={"time_abs": "sample_time_selfreport"})
cortisol_samples = cortisol_samples.drop(columns=["time", "wake_onset_time"])
# use only self-report wake onset information (from "bedtime" dataframe), 
# not merged wake onset information (from cortisol_samples dataframe)
cortisol_samples = cortisol_samples.join(bedtimes).dropna()
cortisol_samples

## Process App Logs

In [ ]:
dict_barcode_day = {}

for subset in tqdm(dataset.groupby("subject")):
    subject_id = subset.index["subject"][0]
    try:
        df_barcode_scanned = process_app_log_single_subject(subset)
        if df_barcode_scanned is not None:
            dict_barcode_day[subject_id] = df_barcode_scanned
    except AppLogDataNotFoundException as e:
        continue

In [ ]:
sample_times = pd.concat(dict_barcode_day, names=["subject"])
sample_times = restructure_sampling_times_dataframe(sample_times)

cortisol_samples_time = cortisol_samples.join(sample_times).join(app_wakeup).sort_index().dropna()
cortisol_samples_time = add_naive_times(cortisol_samples_time)

cortisol_samples_time = sample_times_long_format(cortisol_samples_time)
cortisol_samples_time = cortisol_samples_time.assign(
    **{"time_diff_to_wake_onset": cortisol_samples_time["sample_time"] - cortisol_samples_time["wake_onset"]}
)

cortisol_samples_time

## Compute Cortisol Features For Different Log Types

In [ ]:
# Create copy of dataframe for computing cortisol features and prepare dataframe
cort_samples_compute = cortisol_samples_time.copy()

cort_samples_compute = cort_samples_compute.rename(columns={"time_diff_to_wake_onset": "time"})
cort_samples_compute["time"] = cort_samples_compute["time"].dt.total_seconds() / 60

cort_samples_compute

In [ ]:
cort_feat = compute_saliva_features(cort_samples_compute)
cort_feat

## Export

In [ ]:
export_path = Path("../exports")
export_path.mkdir(exist_ok=True)

In [ ]:
cortisol_samples_time.to_csv(export_path.joinpath("cortisol_samples_log_types.csv"))

In [ ]:
cort_feat.to_csv(export_path.joinpath("cortisol_features_log_types.csv"))